In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import models, layers
import tensorflow.keras as keras
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical

In [3]:


# загрузка тренировочных и тестовых данных
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# конвертация чисел из uint8 в float32
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# нормализация данных [0, 1]
x_train /= 255 
x_test /= 255 

# трансформация лейблов в one-hot encoding
y_train = to_categorical(y_train, 10) 
y_test = to_categorical(y_test, 10) 

# изменение размерности массива в 4D массив
x_train = x_train.reshape(x_train.shape[0], 28,28,1)
x_test = x_test.reshape(x_test.shape[0], 28,28,1)

4423680/4422102 [==============================] - 0s 0us/step


In [7]:
from tensorflow.keras.models import Model 
# инициализация  модели
input1= layers.Input(shape=(28,28,1))
# первый сверточный слой
x1 = layers.Conv2D(6, kernel_size=(7, 7), strides=(1, 1), activation='tanh',  padding="same")(input1)

# второй пуллинговый слой
x2 = layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(x1)

# третий сверточный слой
x3 = layers.Conv2D(16, kernel_size=(7, 7), strides=(1, 1), activation='tanh', padding='valid')(x2)

# четвертый пуллинговый слой
x4 = layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(x3)

# пятый  слой
x5 = layers.Conv2D(120, kernel_size=(3, 3), strides=(1, 1), activation='tanh', padding='valid')(x4)

# сглаживание CNN выхода чтобы можно было его присоединить к полносвязногому слою
x12 = layers.Flatten()(x5)

# шестой полносвязный слой
x13 = layers.Dense(84, activation='tanh')(x12)

# выходной слой с функцией активации softmax
out_x = layers.Dense(10, activation='softmax')(x13)


# Соберем полную модель сети от входа к выходу 
model1 = Model(inputs = input1, outputs = out_x) 
# сделаем несколько промежуточных выходов (через них посмотрим , что происходит в сети) 
model3 = Model(inputs = input1, outputs = x3)
model5 = Model(inputs = input1, outputs = x5)
# компилияция модели
model1.compile(loss=keras.losses.categorical_crossentropy, optimizer='SGD', metrics=["accuracy"])

# Обучаем модель
hist = model1.fit(x=x_train,y=y_train, epochs=5, batch_size=128, validation_data=(x_test, y_test), verbose=1)

test_score = model1.evaluate(x_test, y_test)
print("Test loss {:.4f}, accuracy {:.2f}%".format(test_score[0], test_score[1] * 100))

Epoch 1/5
469/469 [==============================] - 2s 4ms/step - loss: 1.7741 - accuracy: 0.4252 - val_loss: 0.8805 - val_accuracy: 0.6845
Epoch 2/5
469/469 [==============================] - 2s 3ms/step - loss: 0.8070 - accuracy: 0.7127 - val_loss: 0.7183 - val_accuracy: 0.7360
Epoch 3/5
469/469 [==============================] - 2s 3ms/step - loss: 0.6781 - accuracy: 0.7482 - val_loss: 0.6593 - val_accuracy: 0.7543
Epoch 4/5
469/469 [==============================] - 2s 3ms/step - loss: 0.6252 - accuracy: 0.7672 - val_loss: 0.6193 - val_accuracy: 0.7716
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.5901 - accuracy: 0.7796
Test loss 0.5901, accuracy 77.96%


accurcy - 82.8 для базового варианта

ядро - 3, +1 набор слоёв - 78.73

для всех падднигов 1 на 1 - точность 83.05

добавляем ещё слои - 82.37

сократили слои и расширили ядро до 7 на 7 - 85.02 (3 свёрточных, страйд - 1)

прошлый вариант - страйд 2 - 77.96


Вообщем разумно делать неглубокую сеть с большими ядрами. 

Вариант 1. (простой)

- обучить сверточную нейронную сеть в стиле AlexNet (с падением размера ядра свертки и последовательностью блоков свертка-пулинг  (conv-pool)-(conv-pool)-...) на датасете fashion-mnist
- оценить рост точности при увеличении ширины сети (больше ядер)
- оценить рост точности при увеличении глубины сети (больше слоев)
- сравнить с точностью полносвязной сети для этой выборки
    </li>